In [ ]:
import numpy as np
import scipy.integrate as integrate
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

In [ ]:
# Function to transform Galactocentric coordinates to GD-1 coordinates
def transform(tr, rsun=[-8, 0, 0]):
    """
    Transform Galactocentric Cartesian coordinates to the GD-1 stream coordinate system.
    """
    num_stars = len(tr)

    # Galactocentric coords
    x = tr[:,-1,1]
    y = tr[:,-1,2]
    z = tr[:,-1,3]
    
    # Compute relative positions with respect to the Sun
    xrel = np.array(x - rsun[0])
    yrel = np.array(y - rsun[1])
    zrel = np.array(z - rsun[2])

    # Transformation matrix from Galactocentric to GD-1 stream coordinates
    A = np.array([[-0.4776303088, -0.1738432154, 0.8611897727], 
                  [0.510844589, -0.8524449229, 0.111245042], 
                  [0.7147776536, 0.4930681392, 0.4959603976]])

    # Apply transformation
    positions = np.vstack((xrel, yrel, zrel))
    strcoord = np.matmul(A, positions)

    # Compute phi1 and phi2 (longitude and latitude) in the stream coordinate system
    phi1 = np.arctan2(strcoord[1, :], strcoord[0, :])  # Longitude
    phi2 = np.arcsin(strcoord[2, :] / np.sqrt(np.sum(strcoord**2, axis=0)))  # Latitude

    return phi1, phi2

In [ ]:
# Function to calculate D and its derivative with respect to phi1
def distance_derivative_phi1(tr, rsun=[-8, 0, 0]):
    """
    Compute the distance D and its derivative with respect to phi1 using the transform function.
    """
    x = tr[:,-1,1]
    y = tr[:,-1,2]
    z = tr[:,-1,3]
    
    # Relative positions of stars
    xrel = x - rsun[0]
    yrel = y - rsun[1]
    zrel = z - rsun[2]
    
    # Calculate the distance D
    D = np.sqrt(xrel**2 + yrel**2 + zrel**2)
    
    # Get phi1 using the transform function
    phi1, _ = transform(tr, rsun=rsun)
    
    # Compute the derivative of D with respect to phi1
    dD_dx = xrel / D
    dD_dy = yrel / D
    dD_dz = zrel / D
    
    # Derivatives of the GD-1 coordinates with respect to phi1
    dxgd1_dphi1 = -np.sin(phi1)
    dygd1_dphi1 = np.cos(phi1)
    
    # Compute the derivative of D with respect to phi1
    dD_dphi1 = dD_dx * dxgd1_dphi1 + dD_dy * dygd1_dphi1
    
    return D, dD_dphi1

In [ ]:
# Function to define the integrand for s(phi1)
def integrand(phi1_value, tr, rsun=[-8, 0, 0]):
    """
    Computes the integrand for the integral of s(phi1).
    """
    # Calculate D and dD/dphi1 using the distance_derivative_phi1 function
    D, dD_dphi1 = distance_derivative_phi1(tr, rsun=rsun)
    
    # Integrand formula: sqrt(D^2 + (dD/dphi1)^2)
    integrand_value = np.sqrt(D**2 + dD_dphi1**2)
    
    return integrand_value

# Differential equation to solve for the integral of s(phi1)
def integrand_differential_equation(phi1, y, tr):
    """
    Computes the differential equation for the integration of s(phi1).
    """
    return integrand(phi1, tr)

# Function to perform the integration using solve_ivp
def perform_integration(tr):
    """
    Perform integration of s(phi1) using solve_ivp for each value of phi1.
    """
    # Get phi1 values from the transformation function
    phi1_values, _ = transform(tr)
    
    # Array to store the results
    integral_results = np.zeros(len(phi1_values))
    
    # Perform the integration for each phi1 value
    for i, phi1 in enumerate(phi1_values):
        # Integrate up to the current phi1 value
        result = solve_ivp(integrand_differential_equation, [0, phi1], [0], t_eval=[phi1], args=(tr,))
        integral_results[i] = result.y[0][-1]  # Store the result for the current phi1 value
    
    return integral_results

# Example usage
# Example data: tr in the shape (2000, 2, 7) - placeholder data
tr = np.random.randn(2000, 2, 7)

# Perform the integration
integral_results = perform_integration(tr)

# Plot the results
plt.plot(integral_results)
plt.xlabel(r'Index of Stars')
plt.ylabel(r'$s(\phi_1)$')
plt.title('Integration of $s(\phi_1)$ for each star')
plt.show()
